# Traitement des données météo de 2005 à 2009

In [1]:
import pandas as pd
import numpy as np

## Import du dataset

In [2]:
data_meteo = pd.read_csv('../data/decad_2005_2009.csv')
data_meteo.head()

C:\Users\mathi\AppData\Local\Temp\ipykernel_9604\873452805.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_meteo = pd.read_csv('../data/decad_2005_2009.csv')


,__id,LAMBX,LAMBY,DATE,DECADE,PRENEI_DECAD,PRELIQ_DECAD,PRETOTD_DECAD,T_DECAD,EVAP_DECAD,ETP_DECAD,PE_DECAD,SWI_DECAD,SSWI_DECAD,ECOULEMENT_DECAD,DRAINC_DECAD,RUNC_DECAD
0,181,600,24010,200501,1,0.0,18.4,18.4,10.2,12.9,9.7,5.5,0.763,-0.86,0.0,3.7,1.3
1,184,600,24010,200502,1,0.0,21.4,21.4,7.3,6.9,10.7,14.5,0.763,-1.34,0.0,5.0,1.9
2,187,600,24010,200503,1,0.0,19.6,19.6,4.9,14.6,15.7,5.0,0.806,-0.96,0.0,5.8,1.5
3,190,600,24010,200504,1,0.0,13.1,13.1,9.8,20.1,23.7,-7.0,0.704,-1.38,0.0,3.0,0.2
4,193,600,24010,200505,1,0.0,5.6,5.6,11.7,20.9,38.8,-15.3,0.736,0.01,0.0,4.0,0.3


In [3]:
# Affichage des infos
data_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593531 entries, 0 to 593530
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   __id              593531 non-null  object
 1   LAMBX             593531 non-null  object
 2   LAMBY             593531 non-null  object
 3   DATE              593531 non-null  object
 4   DECADE            593531 non-null  object
 5   PRENEI_DECAD      593531 non-null  object
 6   PRELIQ_DECAD      593531 non-null  object
 7   PRETOTD_DECAD     593531 non-null  object
 8   T_DECAD           593531 non-null  object
 9   EVAP_DECAD        593531 non-null  object
 10  ETP_DECAD         593531 non-null  object
 11  PE_DECAD          593531 non-null  object
 12  SWI_DECAD         593531 non-null  object
 13  SSWI_DECAD        538869 non-null  object
 14  ECOULEMENT_DECAD  593531 non-null  object
 15  DRAINC_DECAD      593531 non-null  object
 16  RUNC_DECAD        593531 non-null  obj

In [4]:
# 1) Forcer T_DECAD en type numérique (float), en remplaçant les valeurs non-convertibles par NaN
data_meteo['T_DECAD'] = pd.to_numeric(
    data_meteo['T_DECAD'],
    errors='coerce'
)

# 2) Grouper par DATE et calculer la moyenne
df_meteo = (
    data_meteo
      .groupby('DATE', as_index=False)['T_DECAD']
      .mean()
      .rename(columns={'T_DECAD':'temp_AVG', 'DATE': 'Date'})
)

df_meteo.head()


,Date,temp_AVG
0,200501,6.150528
1,200502,3.262806
2,200503,-0.328597
3,200504,8.405225
4,200505,12.134131


In [5]:
# Calcul du nombre de valeurs manquantes
df_meteo.isna().sum()

Date        0
temp_AVG    1
dtype: int64

In [6]:
# Calcul du nombre de NaN par colonne
df_meteo.isna().sum(axis=0)

Date        0
temp_AVG    1
dtype: int64

In [7]:
# Création d'une colonne Année à partir des 4 premiers caractères de la colonne Date
col_date = 'Date' if 'Date' in df_meteo.columns else 'DATE'

# Conversion en string, en remplaçant les NaN par une chaîne vide
df_meteo[col_date] = df_meteo[col_date].fillna('').astype(str)

# Extraction des 4 premiers caractères et forcer en numérique
# Les non-numériques deviendront NaN
df_meteo['Année'] = pd.to_numeric(
    df_meteo[col_date].str[:4],
    errors='coerce'
)
# 
df_meteo = df_meteo.dropna(subset=['Année'])
df_meteo['Année'] = df_meteo['Année'].astype(int)

df_meteo.head()

,Date,temp_AVG,Année
0,200501,6.150528,2005
1,200502,3.262806,2005
2,200503,-0.328597,2005
3,200504,8.405225,2005
4,200505,12.134131,2005


In [8]:
col_date = 'Date' if 'Date' in df_meteo.columns else 'DATE'

# 2) S’assurer que c’est bien du string (et pas de NaN)
df_meteo[col_date] = df_meteo[col_date].fillna('').astype(str)

# 3) Extraire les deux derniers caractères pour le mois, convertir en numérique
df_meteo['Mois'] = pd.to_numeric(
    df_meteo[col_date].str[-2:],   # ex. "200501" → "01"
    errors='coerce'                # invalides → NaN
)

# 4) (Optionnel) filtrer/vider les lignes où l’extraction a échoué
df_meteo = df_meteo.dropna(subset=['Mois'])
df_meteo['Mois'] = df_meteo['Mois'].astype(int)

df_meteo.head()

,Date,temp_AVG,Année,Mois
0,200501,6.150528,2005,1
1,200502,3.262806,2005,2
2,200503,-0.328597,2005,3
3,200504,8.405225,2005,4
4,200505,12.134131,2005,5


In [9]:
# Création d'une colonne Nom du Mois
mois_fr = {
    1:  'Janvier',
    2:  'Février',
    3:  'Mars',
    4:  'Avril',
    5:  'Mai',
    6:  'Juin',
    7:  'Juillet',
    8:  'Août',
    9:  'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}

# 2) Création de la colonne Nom_Mois
df_meteo['Nom_Mois'] = df_meteo['Mois'].map(mois_fr)

df_meteo.head()

,Date,temp_AVG,Année,Mois,Nom_Mois
0,200501,6.150528,2005,1,Janvier
1,200502,3.262806,2005,2,Février
2,200503,-0.328597,2005,3,Mars
3,200504,8.405225,2005,4,Avril
4,200505,12.134131,2005,5,Mai


In [10]:
# Affichage des types
df_meteo.dtypes

Date         object
temp_AVG    float64
Année         int64
Mois          int64
Nom_Mois     object
dtype: object

In [11]:
# Export en CSV
df_meteo.to_csv('../staged_data/decad_2005_2009_meteo.csv', index=False)